<a href="https://colab.research.google.com/github/Gugale123/CNN-on-MINIST-dataset/blob/main/CNN_on_minist_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install keras_tuner


Importing libraries


In [35]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [36]:
fasion_mnist=keras.datasets.fashion_mnist

In [37]:
(tri,tril),(tei,tel)=fasion_mnist.load_data()

In [38]:
tri=tri/255
tei=tei/255

In [39]:
tri.shape

(60000, 28, 28)

In [40]:
tri=tri.reshape(len(tri),28,28,1)
tei=tei.reshape(len(tei),28,28,1)

In [41]:
tri.shape

(60000, 28, 28, 1)

In [42]:
def build_model(hp):
    model = keras.Sequential([
     keras.layers.Conv2D(
         filters=hp.Int("con_1_fil",min_value=32, max_value=128,step=16),
         kernel_size=hp.Choice('con_2_ker',values=[3,5]),
         activation='relu',
         input_shape=(28,28,1)
     ),
     keras.layers.Conv2D(
         filters=hp.Int("con_2_fil",min_value=32, max_value=128,step=16),
         kernel_size=hp.Choice('con_2_ker',values=[3,5]),
         activation='relu'
     ),
     keras.layers.Flatten(),
     keras.layers.Dense(
         units=hp.Int("dense_1_units",min_value=32, max_value=128,step=16),
         activation='relu'
     ),
     keras.layers.Dense(10,activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])
    return model

In [43]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [44]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='mn')

INFO:tensorflow:Reloading Oracle from existing project output/mn/oracle.json


In [45]:
tuner_search.search(tri,tril,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 01m 06s]
val_accuracy: 0.878000020980835

Best val_accuracy So Far: 0.906166672706604
Total elapsed time: 00h 07m 30s
INFO:tensorflow:Oracle triggered exit


In [46]:
model=tuner_search.get_best_models(num_models=1)[0]

In [47]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 96)        153696    
_________________________________________________________________
flatten (Flatten)            (None, 38400)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                1228832   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 1,384,522
Trainable params: 1,384,522
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.fit(tri,tril,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1854 - accuracy: 0.9312 - val_loss: 0.2599 - val_accuracy: 0.9103
Epoch 5/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1527 - accuracy: 0.9432 - val_loss: 0.2645 - val_accuracy: 0.9100
Epoch 6/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1230 - accuracy: 0.9549 - val_loss: 0.2808 - val_accuracy: 0.9112
Epoch 7/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0987 - accuracy: 0.9625 - val_loss: 0.3339 - val_accuracy: 0.9062
Epoch 8/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0828 - accuracy: 0.9693 - val_loss: 0.3538 - val_accuracy: 0.9080
Epoch 9/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0679 - accuracy: 0.9757 - val_loss: 0.3988 - val_accuracy: 0.9033
Epoch 10/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0562 - accuracy: 0.9792 - val_loss: 0.4219 - val_a